
# Analyze Data Quality with SageMaker Processing Jobs and Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to process and analyze data sets in order to detect data quality issues and prepare them for model training.  

In this notebook we'll use Amazon SageMaker Processing with a library called [Deequ](https://github.com/awslabs/deequ), and leverage the power of Spark with a managed SageMaker Processing Job to run our data processing workloads.

Here is a great blog post on Deequ for more information:  https://aws.amazon.com/blogs/big-data/test-data-quality-at-scale-with-deequ/

![Deequ](img/deequ.png)

![](img/processing.jpg)

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/amazon-reviews-pds/readme.html

### Dataset Columns:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

In [1]:
!pip install pandas==1.0.2

     |████████████████████████████████| 10.1 MB 5.8 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 0.23.0
    Uninstalling pandas-0.23.0:
      Successfully uninstalled pandas-0.23.0


In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

# Create a Docker Container with Spark and our Python Code

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset preprocessing.

Build the example Spark container.

In [3]:
!cat container/Dockerfile

FROM openjdk:8-jre-slim

RUN apt-get update
RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
RUN apt-get clean
RUN rm -rf /var/lib/apt/lists/*

# http://blog.stuart.axelbrooke.com/python-3-on-spark-return-of-the-pythonhashseed
ENV PYTHONHASHSEED 0
ENV PYTHONIOENCODING UTF-8
ENV PIP_DISABLE_PIP_VERSION_CHECK 1

# Install Hadoop
ENV HADOOP_VERSION 3.0.0
ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
ENV HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
ENV PATH $PATH:$HADOOP_HOME/bin
RUN curl -sL --retry 3 \
  "http://archive.apache.org/dist/hadoop/common/hadoop-$HADOOP_VERSION/hadoop-$HADOOP_VERSION.tar.gz" \
  | gunzip \
  | tar -x -C /usr/ \
 && rm -rf $HADOOP_HOME/share/doc \
 && chown -R root:root $HADOOP_HOME

# Install Spark
ENV SPARK_VERSION 2.4.5
ENV SPARK_PACKAGE spark-${SPARK_VERSION}-bin-without-hadoop
ENV SPARK_HOME /usr/spark-${SPARK_VERSION}
ENV SPARK_DIST_CLASSPATH="$HADOOP_H

In [4]:
docker_repo = 'amazon-reviews-spark-analyzer'
docker_tag = 'latest'

In [5]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  3.023MB
Step 1/33 : FROM openjdk:8-jre-slim
8-jre-slim: Pulling from library/openjdk

c2fa59d0: Pulling fs layer 
01647a92: Pulling fs layer 
1af9b241: Pulling fs layer 
Digest: sha256:b8c9a782ad018d37e96eeae213b81977594b4fffb4cae256c55dffe9a5e702b9
Status: Downloaded newer image for openjdk:8-jre-slim
 ---> bf20b099be53
Step 2/33 : RUN apt-get update
 ---> Running in b9478c4476b6
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [189 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7380 B]
Fetched 8339 kB in 1s (6246 kB/s)
Reading package lists...
Removing intermediate container b9478c4476b6
 ---> 568bcfe

Get:21 http://deb.debian.org/debian buster/main amd64 libpython-stdlib amd64 2.7.16-1 [20.8 kB]
Get:22 http://deb.debian.org/debian buster/main amd64 python2 amd64 2.7.16-1 [41.6 kB]
Get:23 http://deb.debian.org/debian buster/main amd64 python amd64 2.7.16-1 [22.8 kB]
Get:24 http://deb.debian.org/debian buster/main amd64 liblocale-gettext-perl amd64 1.07-3+b4 [18.9 kB]
Get:25 http://deb.debian.org/debian buster/main amd64 libpython3.7-minimal amd64 3.7.3-2+deb10u1 [589 kB]
Get:26 http://deb.debian.org/debian buster/main amd64 python3.7-minimal amd64 3.7.3-2+deb10u1 [1736 kB]
Get:27 http://deb.debian.org/debian buster/main amd64 python3-minimal amd64 3.7.3-1 [36.6 kB]
Get:28 http://deb.debian.org/debian buster/main amd64 libmpdec2 amd64 2.4.2-2 [87.2 kB]
Get:29 http://deb.debian.org/debian buster/main amd64 libpython3.7-stdlib amd64 3.7.3-2+deb10u1 [1734 kB]
Get:30 http://deb.debian.org/debian buster/main amd64 python3.7 amd64 3.7.3-2+deb10u1 [330 kB]
Get:31 http://deb.debian.org/debian

Get:119 http://deb.debian.org/debian buster/main amd64 libpython2.7-dev amd64 2.7.16-2+deb10u1 [31.6 MB]
Get:120 http://deb.debian.org/debian buster/main amd64 libpython2-dev amd64 2.7.16-1 [20.9 kB]
Get:121 http://deb.debian.org/debian buster/main amd64 libpython-dev amd64 2.7.16-1 [20.9 kB]
Get:122 http://deb.debian.org/debian buster/main amd64 libpython3.7 amd64 3.7.3-2+deb10u1 [1498 kB]
Get:123 http://deb.debian.org/debian buster/main amd64 libpython3.7-dev amd64 3.7.3-2+deb10u1 [48.4 MB]
Get:124 http://deb.debian.org/debian buster/main amd64 libpython3-dev amd64 3.7.3-1 [20.1 kB]
Get:125 http://deb.debian.org/debian buster/main amd64 libsasl2-modules amd64 2.1.27+dfsg-1+deb10u1 [104 kB]
Get:126 http://deb.debian.org/debian buster/main amd64 libxml2 amd64 2.9.4+dfsg1-7+b3 [687 kB]
Get:127 http://deb.debian.org/debian buster/main amd64 manpages-dev all 4.16-2 [2232 kB]
Get:128 http://deb.debian.org/debian buster/main amd64 publicsuffix all 20190415.1030-1 [116 kB]
Get:129 http://deb

Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-2_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.2-2) ...
Selecting previously unselected package libpython3.7-stdlib:amd64.
Preparing to unpack .../libpython3.7-stdlib_3.7.3-2+deb10u1_amd64.deb ...
Unpacking libpython3.7-stdlib:amd64 (3.7.3-2+deb10u1) ...
Selecting previously unselected package python3.7.
Preparing to unpack .../python3.7_3.7.3-2+deb10u1_amd64.deb ...
Unpacking python3.7 (3.7.3-2+deb10u1) ...
Selecting previously unselected package libpython3-stdlib:amd64.
Preparing to unpack .../libpython3-stdlib_3.7.3-1_amd64.deb ...
Unpacking libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3-minimal (3.7.3-1) ...
Selecting previously unselected package python3.
(Reading database ... 10374 files and directories currently installed.)
Preparing to unpack .../000-python3_3.7.3-1_amd64.deb ...
Unpacking python3 (3.7.3-1) ...
Selecting previously unselected package netbase.
Preparing to 

Selecting previously unselected package libgssapi-krb5-2:amd64.
Preparing to unpack .../049-libgssapi-krb5-2_1.17-3_amd64.deb ...
Unpacking libgssapi-krb5-2:amd64 (1.17-3) ...
Selecting previously unselected package libsasl2-modules-db:amd64.
Preparing to unpack .../050-libsasl2-modules-db_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-modules-db:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libsasl2-2:amd64.
Preparing to unpack .../051-libsasl2-2_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-2:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libldap-common.
Preparing to unpack .../052-libldap-common_2.4.47+dfsg-3+deb10u1_all.deb ...
Unpacking libldap-common (2.4.47+dfsg-3+deb10u1) ...
Selecting previously unselected package libldap-2.4-2:amd64.
Preparing to unpack .../053-libldap-2.4-2_2.4.47+dfsg-3+deb10u1_amd64.deb ...
Unpacking libldap-2.4-2:amd64 (2.4.47+dfsg-3+deb10u1) ...
Selecting previously unselected packag

Selecting previously unselected package libxml2:amd64.
Preparing to unpack .../097-libxml2_2.9.4+dfsg1-7+b3_amd64.deb ...
Unpacking libxml2:amd64 (2.9.4+dfsg1-7+b3) ...
Selecting previously unselected package manpages-dev.
Preparing to unpack .../098-manpages-dev_4.16-2_all.deb ...
Unpacking manpages-dev (4.16-2) ...
Selecting previously unselected package publicsuffix.
Preparing to unpack .../099-publicsuffix_20190415.1030-1_all.deb ...
Unpacking publicsuffix (20190415.1030-1) ...
Selecting previously unselected package python2.7-dev.
Preparing to unpack .../100-python2.7-dev_2.7.16-2+deb10u1_amd64.deb ...
Unpacking python2.7-dev (2.7.16-2+deb10u1) ...
Selecting previously unselected package python2-dev.
Preparing to unpack .../101-python2-dev_2.7.16-1_amd64.deb ...
Unpacking python2-dev (2.7.16-1) ...
Selecting previously unselected package python-dev.
Preparing to unpack .../102-python-dev_2.7.16-1_amd64.deb ...
Unpacking python-dev (2.7.16-1) ...
Selecting previously unselected pac

Setting up libmpc3:amd64 (1.1.0-1) ...
Setting up libatomic1:amd64 (8.3.0-6) ...
Setting up patch (2.7.6-3+deb10u1) ...
Setting up libk5crypto3:amd64 (1.17-3) ...
Setting up libsasl2-2:amd64 (2.1.27+dfsg-1+deb10u1) ...
Setting up libmpx2:amd64 (8.3.0-6) ...
Setting up libubsan1:amd64 (8.3.0-6) ...
Setting up libisl19:amd64 (0.20-2) ...
Setting up libgirepository-1.0-1:amd64 (1.58.3-2) ...
Setting up libssh2-1:amd64 (1.8.0-2.1) ...
Setting up netbase (5.6) ...
Setting up python-pip-whl (18.1-5) ...
Setting up libkrb5-3:amd64 (1.17-3) ...
Setting up libmpdec2:amd64 (2.4.2-2) ...
Setting up libbinutils:amd64 (2.31.1-16) ...
Setting up cpp-8 (8.3.0-6) ...
Setting up libc-dev-bin (2.28-10) ...
Setting up readline-common (7.0-5) ...
Setting up publicsuffix (20190415.1030-1) ...
Setting up libxml2:amd64 (2.9.4+dfsg1-7+b3) ...
Setting up libcc1-0:amd64 (8.3.0-6) ...
Setting up liblocale-gettext-perl (1.07-3+b4) ...
Setting up liblsan0:amd64 (8.3.0-6) ...
Setting up libitm1:amd64 (8.3.0-6) ...


 ---> Running in 7041d5a0d672
Removing intermediate container 7041d5a0d672
 ---> d7afba1e1003
Step 19/33 : ENV PATH $PATH:${SPARK_HOME}/bin
 ---> Running in 23045522b4e7
Removing intermediate container 23045522b4e7
 ---> f02633ec1d60
Step 20/33 : RUN curl -sL --retry 3   "https://archive.apache.org/dist/spark/spark-${SPARK_VERSION}/${SPARK_PACKAGE}.tgz"   | gunzip   | tar x -C /usr/  && mv /usr/$SPARK_PACKAGE $SPARK_HOME  && chown -R root:root $SPARK_HOME
 ---> Running in c771b3242680
Removing intermediate container c771b3242680
 ---> 482afdbb764b
Step 21/33 : ENV PYSPARK_PYTHON=/usr/bin/python3
 ---> Running in 2662e0e9a73d
Removing intermediate container 2662e0e9a73d
 ---> 64685bc65925
Step 22/33 : ENV PATH="/usr/bin:/opt/program:${PATH}"
 ---> Running in 04cf184543e4
Removing intermediate container 04cf184543e4
 ---> af16cc999111
Step 23/33 : ENV YARN_RESOURCEMANAGER_USER="root"
 ---> Running in 4361d318db9b
Removing intermediate container 4361d318db9b
 ---> 44ca47cfa7ab
Step 24/33 

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [6]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-analyzer:latest


Create ECR repository and push docker image

In [7]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [8]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'amazon-reviews-spark-analyzer' does not exist in the registry with id '086401037028'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:086401037028:repository/amazon-reviews-spark-analyzer",
        "registryId": "086401037028",
        "repositoryName": "amazon-reviews-spark-analyzer",
        "repositoryUri": "086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-analyzer",
        "createdAt": 1587838207.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        }
    }
}


In [9]:
!docker tag $docker_repo:$docker_tag $image_uri

In [10]:
!docker push $image_uri

The push refers to repository [086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-analyzer]

986a4154: Preparing 
0ee2b131: Preparing 
ba328002: Preparing 
1eb866e9: Preparing 
6f8023d3: Preparing 
fa33a696: Preparing 
6b9d6e4d: Preparing 
7cc5f91d: Preparing 
eb2ae899: Preparing 
3a42c7f7: Preparing 
a9b45c60: Preparing 
3b6fbb91: Preparing 
ac5e9bd9: Preparing 
0d58a380: Preparing 
3a42c7f7: Pushed   490.3MB/481.8MBlatest: digest: sha256:c1af8fb33e4e6714d333e314ee0afa7114842afee98a17fdca13b602de82e72a size: 3472


# Run our Analysis Job as a SageMaker Processing Job

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built with our Spark script.

## Review the Spark preprocessing script.

In [11]:
!cat preprocess-deequ.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args['s3_input_data'].replace('s3://', 's3a://')
    print(s3_input_data)
    s3_output_analyze_data = args['s3_output_analyze_data'].replace('s3://', 's3a://')
    print(s3_output_analyze_data)
    
    spark = SparkSession.builder \
        .appName("Amazon_Reviews_Spark_Analyzer") \
        .getOrCreate()

    # Invoke Main from preprocess-deequ.jar
    getattr(spark._jvm.SparkAmazonReviewsAnalyzer, "run")(s3_input_data, s3_output_analyze_data)

if __name__ == "__main__":
    main()


In [12]:
!cat deequ/preprocess-deequ.scala

import com.amazon.deequ.analyzers.runners.{AnalysisRunner, AnalyzerContext}
import com.amazon.deequ.analyzers.runners.AnalyzerContext.successMetricsAsDataFrame
import com.amazon.deequ.analyzers.{Compliance, Correlation, Size, Completeness, Mean, ApproxCountDistinct}
import com.amazon.deequ.{VerificationSuite, VerificationResult}
import com.amazon.deequ.VerificationResult.checkResultsAsDataFrame
import com.amazon.deequ.checks.{Check, CheckLevel}
import com.amazon.deequ.suggestions.{ConstraintSuggestionRunner, Rules}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}


object SparkAmazonReviewsAnalyzer {
  def run(s3InputData: String, s3OutputAnalyzeData: String): Unit = {
    // TODO:  Retrieve s3_input_data and s3_output_analyze_data args from the list of args (argv-style)
//    System.out.println(args)
    System.out.println(s"s3_input_data: ${s3InputData}")
    System.out.p

In [13]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-analyzer',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
                            env={
                                'mode': 'jar',
                                'main_class': 'Main'
                            })

In [14]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-west-2-086401037028/amazon-reviews-pds/tsv/


In [15]:
!aws s3 ls $s3_input_data

2020-04-25 16:31:58   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-04-25 16:32:07   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


## Setup Output Data

In [16]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)
processing_job_name = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2020-04-25-18-10-34


In [17]:
s3_output_analyze_data = 's3://{}/{}/output'.format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-west-2-086401037028/amazon-reviews-spark-analyzer-2020-04-25-18-10-34/output


## Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [18]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-deequ.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_analyze_data', s3_output_analyze_data,
              ],
              # See https://github.com/aws/sagemaker-python-sdk/issues/1341 for why we need to specify a dummy-output
              outputs=[
                  ProcessingOutput(s3_upload_mode='EndOfJob',
                                   output_name='dummy-output',
                                   source='/opt/ml/processing/output')
              ],
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [19]:
processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, processing_job_name)))

In [20]:
from IPython.core.display import display, HTML

s3_job_output_prefix = output_prefix

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, s3_job_output_prefix, region)))

# Please Wait Until the Processing Job Completes!

In [21]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-086401037028/spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]}, 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2020-04-25-18-10-34-581', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.r5.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '086401037028.dkr.ecr.us-west-2.amazonaws.com/amazon-revie

# Please Wait Until the Processing Job Completes Before Continuing!

# Inspect the Processed Output (Quality Checks on our Dataset)
Take a look at a few rows of the transformed dataset to make sure the preprocessing was successful.

In [22]:
!aws s3 ls --recursive $s3_output_analyze_data/

## Copy the Output from S3 to Local
* dataset-metrics/
* constraint-checks/
* success-metrics/
* constraint-suggestions/


In [23]:
!aws s3 cp --recursive $s3_output_analyze_data ./amazon-reviews-spark-analyzer/ --exclude="*" --include="*.csv"

## Analyze Constraint Checks

In [24]:
import glob
import pandas as pd
import os

def load_dataset(path, sep, header):
    data = pd.concat([pd.read_csv(f, sep=sep, header=header) for f in glob.glob('{}/*.csv'.format(path))], ignore_index = True)

    return data

In [25]:
df_constraint_checks = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-checks/', sep='\t', header=0)
df_constraint_checks[['check', 'constraint', 'constraint_status', 'constraint_message']]

ValueError: No objects to concatenate

## Analyze Dataset Metrics

In [ ]:
df_dataset_metrics = load_dataset(path='./amazon-reviews-spark-analyzer/dataset-metrics/', sep='\t', header=0)
df_dataset_metrics

## Analyze Success Metrics

In [ ]:
df_success_metrics = load_dataset(path='./amazon-reviews-spark-analyzer/success-metrics/', sep='\t', header=0)
df_success_metrics

## Analyze Constraint Suggestions

In [ ]:
df_constraint_suggestions = load_dataset(path='./amazon-reviews-spark-analyzer/constraint-suggestions/', sep='\t', header=0)
df_constraint_suggestions.columns=['column_name', 'description', 'code']
df_constraint_suggestions